# 서울시 공공 자전거 실시간 대여 정보

In [128]:
import requests
import json
import pandas as pd 
import folium


from folium.plugins import MarkerCluster
from pandas.io.json import json_normalize
from ipywidgets import interact

###### 변수 정의 및 데이터 불러오기
1. 서울 열린 데이터 광장 - 서울시 공공 자전거 실시간 대여 정보
http://data.seoul.go.kr/dataList/OA-15493/A/1/datasetView.do
2. 행정구별 좌표

In [129]:
### 자전거 대여 정보
key = "6c586f625479686935306a575a4e53"
bike_type = "json"
station_url = "xml"
start_1 = "1"
end_1 = "1000"
start_2 = "1001"
end_2 = "2000"
start_3 = "2001"
end_3 = "3000"

bike_url_1 = "http://openapi.seoul.go.kr:8088/{}/{}/bikeList/{}/{}/".format(key, bike_type, start_1, end_1)
bike_url_2 = "http://openapi.seoul.go.kr:8088/{}/{}/bikeList/{}/{}/".format(key, bike_type, start_2, end_2)
bike_url_3 = "http://openapi.seoul.go.kr:8088/{}/{}/bikeList/{}/{}/".format(key, bike_type, start_3, end_3)

### 행정구별 좌표
seoul = pd.read_csv("Desktop/code_it/seoul.csv")

cities = seoul['이름']
seoul = pd.read_csv("Desktop/code_it/seoul.csv")
seoul_dict = seoul.set_index('이름').T.to_dict('list')

###### json -> dataframe
한번에 최대 1000건 호출 가능. 나눠서 호출

In [130]:
### 서울 열린데이터 광장 실시간 대여정보 api 불러오기 및 데이터프레임

response_1 = requests.get(bike_url_1)
bike_dic_1 = response_1.json()
bike_json_1 = bike_dic_1['rentBikeStatus']['row']

response_2 = requests.get(bike_url_2)
bike_dic_2 = response_2.json()
bike_json_2 = bike_dic_2['rentBikeStatus']['row']

response_3 = requests.get(bike_url_3)
bike_dic_3 = response_3.json()
bike_json_3 = bike_dic_3['rentBikeStatus']['row']

bike = pd.json_normalize(bike_json_1+bike_json_2+bike_json_3)

###### folium - marker, cluster, 행정구별 검색 생성

In [151]:
@interact(city=cities)
def show_canned_examples(city):
    
    latlon_for_city = seoul_dict
    zoom = 0
    
    if city == '서울':
        zoom = 11
    else :
        zoom = 14
    map = folium.Map(location=latlon_for_city[city], zoom_start=zoom)
    
    marker_cluster = MarkerCluster().add_to(map)
    
    for name, parking, rack, lat, lon in zip(bike.stationName, bike.parkingBikeTotCnt, bike.rackTotCnt, bike.stationLatitude,  bike.stationLongitude):
        popup_message = "정류장 이름 : {} <br>거치 가능한 공간 : {} <br>대기중인 자전거 : {}".format(name, rack, parking)
        iframe = folium.IFrame(popup_message)
        popup = folium.Popup(iframe, min_width=400, max_width=400)
        folium.Marker([lat, lon], tooltip = name, popup = popup, icon = folium.Icon(color='red', icon='flag')).add_to(marker_cluster)
    
    return map




interactive(children=(Dropdown(description='city', options=('서울', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중…

In [152]:
interact(show_canned_examples, city = "서울")

interactive(children=(Text(value='서울', description='city'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_canned_examples(city)>